In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.core.display import display,Image
from string import Template
import pandas as pd
import numpy as np
import IPython.display
import warnings 

In [ ]:
#### Defining Convolution layer and its backpropagatoin function 

#### Back propagation in Conv layer is a full convolution of gradients with output


class Conv:
    def __init__(self, num_filters, input_shape):
        
        self.num_filters = num_filters
        self.channels = input_shape[2]
        self.filters = np.random.randn(self.num_filters,3, 3,self.channels)
        self.check = []
        
        #print(self.channels)
    
    def conv_operation(self, inp):
        
#         if len(inp.shape)>2:
            
#             self.channels = inp.shape[2]
#         else:
#             self.channels = None
        
        for i in range(inp.shape[0] -2):
            for j in range(inp.shape[1]-2):
                
                region = inp[i:i+3,j:j+3,:]
                #print(region.shape)
                
                yield i , j , region
                
    def conv2d(self, input):
        
        #print(self.filters)
        self.last_input = input
        
        l,w = input.shape[0], input.shape[1]

        output = np.zeros((l -2, w-2,self.num_filters))
        
        for i, j, region in self.conv_operation(input):
            
            output[i,j] = np.sum(region*self.filters, axis = (1,2,3))
            #print(output.shape)
            
        return output  

    def back_prop(self, dl_do, lr=0.1):
        
        
        dl_do_filters = np.zeros(self.filters.shape)
        
        for i, j ,region in self.conv_operation(self.last_input):
            
            for f in range(self.num_filters):
                #print(region.shape)
                
                dl_do_filters[f] += dl_do[i,j,f]*region
                
                #print(region.shape)
                
        self.filters -= lr*dl_do_filters
        
        self.check.append(self.filters)
        
        return None     

In [ ]:
### There are no weights for maxpool layer but backpropagation is performed for Maximum activated neuron to 
#### propagate gradient to the previous layer

class MaxPool:
    
    def pool_operation(self, image):
        
        new_h = image.shape[0]//2
        new_w = image.shape[1]//2
        
        for i in range(new_h):
            for j in range(new_w):
                pool_region = image[i*2:i*2+2, j*2:j*2+2]
                
                yield i , j ,pool_region
                
    def forward_pool(self, input):
        self.last_input = input
        
        output = np.zeros((input.shape[0]//2, input.shape[1]//2, input.shape[2]))
        
        for i , j , pool_region in self.pool_operation(input):
            
            output[i,j] = np.amax(pool_region, axis = (0,1))
            
        return output
    def back_prop(self, dl_do, lr=0.1):
        
        dl_do_input = np.zeros(self.last_input.shape)
        
        for i, j , pool_region in self.pool_operation(self.last_input):
            
            h,w,f = pool_region.shape
            amax = np.amax(pool_region, axis = (0,1))
            
            for ii in range(h):
                for jj in range(w):
                    for ff in range(f):
                        
                        if pool_region[ii,jj,ff] == amax[ff]:
                            
                            dl_do_input[ii*2:ii*2+2, jj*2:jj*2+2, ff] = dl_do[ii,jj,ff]
                            break
        return dl_do_input    

In [ ]:

#### Defining softmax and its derivative for backpropagation 

class Softmax:
    
    def __init__(self, input_len, nodes):
        
        self.weights = np.random.randn(input_len, nodes)
        self.biases = np.random.randn(nodes)
    
    
    def forward_soft(self, input):
        
        self.last_input_shape = input.shape
        
        input = input.flatten()
        
        self.last_input = input
        
        input_len, nodes = self.weights.shape
        
        output = np.dot(input, self.weights)+ self.biases
        
        self.last_output =  output
        
        exp = np.exp(output)
        
        return exp/(np.sum(exp, axis = 0))
    
    
    
    
    def back_prop_soft(self, dl_do, lr=0.1):
        
        #print(dl_do)
        
        for i in range(len(dl_do)):
            
            if dl_do[i] == 0:
                continue
            exp_out = np.exp(self.last_output)
            total_out = np.sum(exp_out)
            
            #print(exp_out.shape)
            do_dt = -exp_out[i]*exp_out/(total_out**2)
            do_dt[i] = exp_out[i]*(total_out - exp_out[i])/(total_out**2)
            
            dt_dw = self.last_input
            dt_db = 1
            dt_dx = self.weights
            
            ## dl_dw = dl_do*do_dt*dt_dw
            dl_dt = dl_do[i]*do_dt
            #print(dl_do.shape)
            #print(dl_dt.shape)
            dl_dw = dt_dw[np.newaxis].T@dl_dt[np.newaxis]
            dl_db = dl_dt*dt_db
            dl_dx = dt_dx@dl_dt
            
            ## update weights and biases
#             print(dl_dw.shape)
#             print(self.weights.shape)
            self.weights += lr*dl_dw
            self.biases += lr*dl_db
            
            return dl_dx.reshape(self.last_input_shape)       

In [ ]:
from tensorflow.keras.datasets import mnist

(train_X, train_y), (test_X, test_y) = mnist.load_data()

train_images = train_X[:1000]
train_images = np.array([np.expand_dims(i,-1) for i in train_images])
train_labels = train_y[:1000]
test_images = test_X[:1000]
test_labels = test_y[:1000]

In [ ]:
train_X.shape, train_y.shape

In [ ]:
conv = Conv(8,(28,28,1))
pool = MaxPool()
soft = Softmax(13*13*8,10)


def model(image, label):
    
    conv1 = conv.conv2d(image/255.0-0.5)
    po = pool.forward_pool(conv1)
    out = soft.forward_soft(po)
    
    ### Calculate Cross Entropy ####
    
    loss = -np.log(out[label])
    acc = 1 if(np.argmax(out) == label) else 0
    
    
    return out, loss, acc
    
    

def train(image, label, lr = 0.1, epoch = 5):
    
    
    out, loss, acc = model(image, label)
    
    
    #print(out[label])
    
    gradient = np.zeros(out.shape)
    
    gradient[label] = -1/out[label]
    
    ########## Backpropagation########
    
    
    error = soft.back_prop_soft(dl_do = gradient, lr = 0.4)

    error = pool.back_prop(error)

    error = conv.back_prop(error, lr)
    
    return loss, acc


In [ ]:
epochs = 5
for epoch in range(epochs):
    
    print('----EPOCH %d ---'%(epoch+1))
    
    accc = 0
    loss = 0
    
    for i , (image, label) in enumerate(zip(train_images, train_y)):
        
        if(i>0 and i %100 == 99):
            print('[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%' %(i + 1, loss / 100, accc))
            
            loss = 0
            accc = 0
            
        l, acc = train(image, label)
        loss+= l
        acc += accc
            
            
            
        
        
    
    

In [ ]:
im = np.expand_dims(train_X[0], -1)

In [ ]:
conv = Conv(8, im.shape)

In [ ]:
conv.check[0]

In [ ]:
plt.imshow(conv.conv2d(im)[:,:,8])

In [ ]:
t = np.random.randn(3,3,1)

In [ ]:
t[0,0,:].shape